In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os 
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [3]:
client = bigquery.Client()

In [6]:
query = """SELECT ANO as ano, MES as mes,TIPO_APOSENTADORIA as tipo_aposentadoria, CASE WHEN ORGSUP_LOTACAO ="Sem informação" Then ORG_LOTACAO
else ORGSUP_LOTACAO end as org_lotacao, 1 as total
 FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_mar_siape_aposentados_cadastro` 
union all
SELECT ANO as ano, MES as mes,TIPO_APOSENTADORIA as tipo_aposentadoria, CASE WHEN ORGSUP_LOTACAO ="Sem informação" Then ORG_LOTACAO
else ORGSUP_LOTACAO end as org_lotacao, 1 as total
 FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2021_mar_siape_aposentados_cadastro` 
union all
SELECT ANO as ano, MES as mes,TIPO_APOSENTADORIA as tipo_aposentadoria, CASE WHEN ORGSUP_LOTACAO ="Sem informação" Then ORG_LOTACAO
else ORGSUP_LOTACAO end as org_lotacao, 1 as total
 FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2022_mar_siape_aposentados_cadastro` 
union all
SELECT ANO as ano, MES as mes,TIPO_APOSENTADORIA as tipo_aposentadoria, CASE WHEN ORGSUP_LOTACAO ="Sem informação" Then ORG_LOTACAO
else ORGSUP_LOTACAO end as org_lotacao, 1 as total
 FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_mar_siape_aposentados_cadastro` 
"""  

In [8]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,tipo_aposentadoria,org_lotacao,total
0,2023,3,APOSENTADORIA VOLUNTARIA,Ministério do Turismo,1
1,2023,3,APOSENTADORIA VOLUNTARIA,Ministério da Economia,1
2,2023,3,APOSENTADORIA VOLUNTARIA,Ministério da Economia,1
3,2023,3,APOSENTADORIA VOLUNTARIA,Ministério da Economia,1
4,2023,3,APOSENTADORIA VOLUNTARIA,Ministério da Economia,1
...,...,...,...,...,...
1695334,2022,3,APOSENTADORIA VOLUNTARIA,Ministério da Economia,1
1695335,2022,3,APOSENTADORIA VOLUNTARIA,Ministério da Economia,1
1695336,2022,3,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,1
1695337,2022,3,APOSENTADORIA VOLUNTARIA,Ministério da Defesa,1


In [10]:
df.columns

Index(['ano', 'mes', 'tipo_aposentadoria', 'org_lotacao', 'total'], dtype='object')

In [11]:
df['ano'].unique()

<IntegerArray>
[2023, 2021, 2020, 2022]
Length: 4, dtype: Int64

In [12]:
df['org_lotacao'].unique()

array(['Ministério do Turismo', 'Ministério da Economia',
       'Ministério da Saúde', 'Ministério da Defesa',
       'Ministério da Educação', 'Ministério de Minas e Energia',
       'Ministério da Justiça e Segurança Pública',
       'Ministério da Infraestrutura',
       'Ministério da Ciência, Tecnologia, Inovações e Comunicações',
       'Presidência da República', 'Ministério das Relações Exteriores',
       'Ministério da Agricultura, Pecuária e Abastecimento',
       'Ministério do Desenvolvimento Regional',
       'Ministério do Meio Ambiente',
       'Ministério do Desenvolvimento Social',
       'Controladoria-Geral da União', 'MINISTERIO DAS COMUNICACOES',
       'Defensoria Pública da União',
       'Ministério da Mulher, Família e Direitos Humanos',
       'Caixa de Financiamento Imobiliário da Aeronáutica'], dtype=object)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Id_SERVIDOR_PORTAL                    1000 non-null   Int64  
 1   NOME                                  1000 non-null   object 
 2   CPF                                   1000 non-null   object 
 3   MATRICULA                             1000 non-null   object 
 4   COD_TIPO_APOSENTADORIA                1000 non-null   Int64  
 5   TIPO_APOSENTADORIA                    1000 non-null   object 
 6   DATA_APOSENTADORIA                    1000 non-null   object 
 7   DESCRICAO_CARGO                       1000 non-null   object 
 8   COD_UORG_LOTACAO                      1000 non-null   Int64  
 9   UORG_LOTACAO                          1000 non-null   object 
 10  COD_ORG_LOTACAO                       1000 non-null   Int64  
 11  ORG_LOTACAO       

In [ ]:
df.to_csv('', sep='', encoding='latin-1', index=False)

In [ ]:
os.chdir('')

-----------------------------------------------------------

Testes

In [6]:
df = pd.read_csv("ESTADIC_lei_plano_carreira_por_ano.csv", encoding="latin-1", sep=';', decimal=',')
df

,ano,sigla_uf,cod_uf,uf,ano_criacao_lei,periodo_anual,lei_criacao_plano_carreira
0,2021,RJ,33,Rio De Janeiro,1990,Década de 90,1614
1,2021,SP,35,São Paulo,1997,Década de 90,836
2,2021,PE,26,Pernambuco,1998,Década de 90,11559
3,2021,ES,32,Espírito Santo,1998,Década de 90,5580
4,2021,MT,51,Mato Grosso,1998,Década de 90,50
5,2021,MS,50,Mato Grosso Do Sul,2000,Década de 90,87
6,2021,SE,28,Sergipe,2001,Década de 2000,61
7,2021,GO,52,Goiás,2001,Década de 2000,13909
8,2021,BA,29,Bahia,2002,Década de 2000,8261
9,2021,PB,25,Paraíba,2003,Década de 2000,7419


Automatizando troca de csv

In [45]:
##os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [46]:
## baixando os atuais com , e exportando com ";"

In [47]:
## listando todos os itens do diretório
##len(glob.glob('*.csv'))

In [48]:
## Serve apenas na primeira vez . 

##for i in list(range(0,len(glob.glob('*.csv')))):
    
   # df= pd.read_csv(glob.glob('*.csv')[i])
    #df.to_csv(glob.glob('*.csv')[i],index=False, encoding="latin-1", sep=';', decimal=',')